# Read Data

In [1]:
import hickle as hkl

In [2]:
import numpy as np

In [13]:
#获得ｋｍｅｒ的函数
d2 = {'a':0, 'A':0, 'g':1, 'G':1, 'c':2, 'C':2, 't':3, 'T':3, 'N':0, 'n':0}
def seq_to_kspec(seq):
    mat = np.zeros((4096,1))
    k = 0
    if len(seq)<6:
        return mat
    for i in range(6):
        k = k*4 + d2[seq[i]]
    mat[k]+=1  
    for i in range(6,len(seq)):
        k = k - 4**5*d2[seq[i-6]]
        k = k*4 + d2[seq[i]]
        mat[k] += 1  
    return mat

In [6]:

labels=hkl.load('./Gen_data/15_label.hkl')
ProA =hkl.load('./Gen_data/15_A0.hkl')
ProB= hkl.load('./Gen_data/15_B0.hkl')

In [20]:
PESeq =dict()
PESeq['label'] =[]
PESeq['ProA-Seq'] =[]
PESeq['ProB-Seq'] = []
PESeq['ProA-Kmer']=[]
PESeq['ProB-Kmer']=[]
PESeq['index']=[]

    

In [21]:
for index in range(len(labels)):
    PESeq['label'].append(labels[index])
    PESeq['ProA-Seq'].append(ProA[index])
    PESeq['ProB-Seq'].append(ProB[index])
    PESeq['ProA-Kmer'].append(seq_to_kspec(ProA[index]))
    PESeq['ProB-Kmer'].append(seq_to_kspec(ProB[index]))
    PESeq['index'].append(index)
    

In [22]:
# Get X,Y with   limite  making positive and negative samples 's numbers is the same
X=[]
Y=[]
for index in range( len(PESeq['label'])):
    Y.append(PESeq['label'][index])
    X.append(np.append(PESeq['ProA-Kmer'][index],PESeq['ProB-Kmer'][index],axis=1))
print np.shape(X)
print np.shape(Y)
X = np.array(X)
Y  = np.array(Y)


index= range(len(PESeq['label']))

(18698, 4096, 2)
(18698,)


In [23]:
from keras.utils.np_utils import to_categorical

Using Theano backend.
WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: CUDA driver version is insufficient for CUDA runtime version)


In [24]:

VALIDATION_SPLIT =0.2

np.random.seed(1234)

np.random.shuffle(index)
nb_validation_samples = int(VALIDATION_SPLIT*len(Y))

In [25]:

x_train = X[index[: - nb_validation_samples]]
y_train = Y[index[:-nb_validation_samples]]
x_val = X[index[-nb_validation_samples:]]
y_val =Y[index[-nb_validation_samples:]]



In [26]:
y_train = to_categorical(y_train) #one-hot 编码labels
#y_val 其实没必要 one-hot编码
y_val = to_categorical(np.asarray(y_val))

In [27]:
print 'x_train_shape:',np.shape(x_train)
print 'y_train_shape:',np.shape(y_train)

x_train_shape: (14959, 4096, 2)
y_train_shape: (14959, 2)


In [28]:
import hickle as hkl

In [29]:
hkl.dump(x_val,'./Gen_data/15_x_val.hkl')
hkl.dump(y_val,'./Gen_data/15_y_val.hkl')
hkl.dump(x_train,'./Gen_data/15_x_train.hkl')
hkl.dump(y_train,'./Gen_data/15_y_train.hkl')

In [1]:
import hickle as hkl
x_val = hkl.load('./Gen_data/15_x_val.hkl')
y_val = hkl.load('./Gen_data/15_y_val.hkl')
x_train = hkl.load('./Gen_data/15_x_train.hkl')
y_train = hkl.load('./Gen_data/15_y_train.hkl')

# Make Network

In [2]:
from keras.layers import Embedding
from keras.layers import Dense,Input,Activation
from keras.layers import Embedding, LSTM, Bidirectional,GRU,InputLayer
from keras.models import Model,Sequential
from  keras.regularizers import ActivityRegularizer
from keras.layers.core import Dropout,Flatten

Using Theano backend.
WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: CUDA driver version is insufficient for CUDA runtime version)


In [3]:
from keras import layers

In [18]:
kmer_input = Input(shape=(4096,2), dtype='float32')
flat = Flatten()(kmer_input)

dense1 = Dense(64)(flat)

dense2 =Dense(32)(dense1)

preds = Dense(len( y_train[0]), activation='softmax',activity_regularizer= ActivityRegularizer(l2=0.005))(dense2)

model  = Model (kmer_input,preds)

In [16]:
print x_val[:5]

[[[  0.  11.]
  [  0.   1.]
  [  2.   1.]
  ..., 
  [  0.   0.]
  [  0.   1.]
  [  0.   0.]]

 [[  1.   1.]
  [  1.   1.]
  [  1.   2.]
  ..., 
  [  0.   0.]
  [  2.   2.]
  [  1.   0.]]

 [[  5.   0.]
  [  3.   0.]
  [  1.   0.]
  ..., 
  [  0.   3.]
  [  1.   2.]
  [  0.   1.]]

 [[  3.   0.]
  [  2.   2.]
  [  1.   0.]
  ..., 
  [  2.   1.]
  [  1.   1.]
  [  0.   2.]]

 [[  0.   1.]
  [  0.   1.]
  [  0.   0.]
  ..., 
  [  2.   0.]
  [  0.   0.]
  [  1.   0.]]]


In [19]:
model.compile(loss='mse',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - attention LSTM network")
model.summary()


model fitting - attention LSTM network
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 4096, 2)       0                                            
____________________________________________________________________________________________________
flatten_2 (Flatten)              (None, 8192)          0           input_2[0][0]                    
____________________________________________________________________________________________________
dense_4 (Dense)                  (None, 64)            524352      flatten_2[0][0]                  
____________________________________________________________________________________________________
dense_5 (Dense)                  (None, 32)            2080        dense_4[0][0]                    
____________________________________________________

In [21]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), nb_epoch=50, batch_size=100)

Train on 14959 samples, validate on 3739 samples
Epoch 1/50
14959/14959 [==============================] - 11s - loss: 0.9958 - acc: 0.5020 - val_loss: 0.9575 - val_acc: 0.5231
Epoch 2/50
14959/14959 [==============================] - 9s - loss: 1.0002 - acc: 0.4987 - val_loss: 0.9916 - val_acc: 0.5052
Epoch 3/50
14959/14959 [==============================] - 7s - loss: 1.0012 - acc: 0.4974 - val_loss: 0.9916 - val_acc: 0.5052
Epoch 4/50
14959/14959 [==============================] - 7s - loss: 0.9988 - acc: 0.5000 - val_loss: 1.0014 - val_acc: 0.4953
Epoch 5/50
14959/14959 [==============================] - 8s - loss: 0.9955 - acc: 0.5026 - val_loss: 0.9674 - val_acc: 0.5162
Epoch 6/50
14959/14959 [==============================] - 7s - loss: 0.9779 - acc: 0.5147 - val_loss: 1.0018 - val_acc: 0.4948
Epoch 7/50
14959/14959 [==============================] - 12s - loss: 0.9977 - acc: 0.5015 - val_loss: 1.0020 - val_acc: 0.4948
Epoch 8/50
14959/14959 [==============================] - 8s

In [40]:
pro = model.predict(x_val)

In [41]:
y_pred = [  1 if item1>item0 else 0 for item0,item1 in pro]

In [42]:
from sklearn import metrics

In [43]:
acc = metrics.accuracy_score(y_val[:,1],y_pred)
print acc

auc = metrics.roc_auc_score(y_val[:,1],pro[:,1])
print auc

f1_score = metrics.f1_score(y_val[:,1],y_pred)
print f1_score

0.505215298208
0.5
0.0


/home/yinqijin/.local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


# Save&Search

In [ ]:
dir(model)

In [ ]:
model.__dict__

In [ ]:
model.save_weights('./Gen_data/15_Dense.h5')

In [ ]:
model.load_weights('./Gen_data/15_Dense.h5',by_name=True)